In [441]:
# Here we are importing all the required libraries
import cv2
import numpy as np
import sys

In [442]:
# This class contains all the required methods
class GraphSegmentation:
    def __init__(self):
        # Here we are creating a dictionary that store 1-D to 2-D index mapping
        # ex: 0 -> (0, 0) or 12 -> (3, 4)
        # It is helpful when we convert graph back to image
        self.vertex_to_img = {}

    def gaussian_filter(self, img, ksize, sigma):
        # Here we are creating gaussian kernel and then using convolution to smooth out the image.
        nimage = np.float64(img)
        kpad = int((ksize-1)/2)
        # Here we are creating a gaussian kernel of size (ksize x ksize)
        kernel = np.zeros((ksize, ksize), dtype='float64')
        # Here we are using loop to apply convolution
        for i in range(ksize):
            for j in range(ksize):
                kernel[i, j] = ( 1/np.sqrt(2*np.pi*(sigma**2)) ) * np.exp(- ( ( ((i-kpad)**2) + ((j-kpad)**2) )/( 2*(sigma**2) ) ) )
        ksum = np.sum(kernel)
        kernel /= ksum
        blur_image = cv2.filter2D(nimage, -1, kernel)
        return np.uint8(blur_image)

    def image_to_grid_graph(self, img):
        # We are using this method for Image To Graph Conversion
        img = np.int64(img)
        height, width = img.shape
        # This list store all the nodes and there connected neighbour node
        G = []
        # This list store all the edges of the graph along with their weights
        E = []
        # Using this loop we iterate over each pixel and convert it to a node.
        for row in range(height):
            for col in range(width):
                # Here we are doing book-keeping we store each row major order index along with its 2D index
                self.vertex_to_img[row*width+col] = (row, col)
                # Here we are calling a method to find neighbour of any pixel
                G, E = self.check_8_neigh(G, E, row, col, height, width, img)
        # This method is returnig a Adjacency List Graph and Edge List Graph
        return np.array(G), np.array(E)

    def check_8_neigh(self, G, E, row, col, height, width, img):
        # We are using this method for checking 8-neighbour connectivity
        # 1. We check boundary conditions.
        # 2. We calculate edge weight, which is difference between current pixel 
        #    intensity and neighbour pixel intensity
        # 3. We add neighbour node to adjacency list along with edge weight between them
        # 4. We add edge to Edge list.
        # This list store all the node connected to current node
        l = []
        if (row-1)>=0 and (col-1)>=0:
            w = abs(img[row, col]-img[row-1, col-1])
            l.append([(row-1)*width+(col-1), w])
            E.append([row*width+col , (row-1)*width+(col-1), w])
        if (row-1)>=0:
            w = abs(img[row, col]-img[row-1, col])
            l.append([(row-1)*width+col, w])
            E.append([row*width+col , (row-1)*width+col, w])
        if (row-1)>=0 and (col+1)<width:
            w = abs(img[row, col]-img[row-1, col+1])
            l.append([(row-1)*width+(col+1), w])
            E.append([row*width+col , (row-1)*width+(col+1), w])
        if (col-1)>=0:
            w = abs(img[row, col]-img[row, col-1])
            l.append([row*width+(col-1), w])
            E.append([row*width+col , row*width+(col-1), w])
        if (col+1)<width:
            w = abs(img[row, col]-img[row, col+1])
            l.append([row*width+(col+1), w])
            E.append([row*width+col , row*width+(col+1), w])
        if (row+1)<height and (col-1)>=0:
            w = abs(img[row, col]-img[row+1, col-1])
            l.append([(row+1)*width+(col-1), w])
            E.append([row*width+col , (row+1)*width+(col-1), w])
        if (row+1)<height:
            w = abs(img[row, col]-img[row+1, col])
            l.append([(row+1)*width+col, w])
            E.append([row*width+col , (row+1)*width+col, w])
        if (row+1)<height and (col+1)<width:
            w = abs(img[row, col]-img[row+1, col+1])
            l.append([(row+1)*width+(col+1), w])
            E.append([row*width+col , (row+1)*width+(col+1), w])
        G.append(l)
        return G, E

    def find_comp(self, S, node):
        # This method is used to find parent node of each connected component
        # Here Path Compression algorithm is also used
        while node!=S[node]:
            S[node] = S[S[node]]
            node = S[node]
        return S, node
    
    def build_mst(self, G, E):
        # We are using this method to build Minimum spanning tree using image graph
        V =  G.shape[0]
        # This list store parent component of each node, Parent List
        S = []
        # We are using this list to store rank of each component which is part of 
        # building mst algorithm, in easy words it store how many nodes are present in
        # each connected component.
        R = []
        # This dictionary store minimum spanning tree build using image graph
        MST = {}
        # Here we are initalizing default values to each list
        # In begning each node is parent of itself and rank of each component is zero
        for i in range(V):
            S.append(i)
            R.append(0)
        # Here we are sorting edge list based on edge weight in ascending order
        E = sorted(E, key=lambda x: x[2], reverse=False)
        # This dictionary store connected component
        comp_di = {}
        # In this loop we are iterating over each edge and building MST
        for i,edge in enumerate(E):
            sys.stdout.write('\rbuilding mst : {0}%'.format(int((float(i)/len(E))*100)))
            u, v, w = edge
            # Here we are finding parent component of node u
            S, c1 = self.find_comp(S, u)
            # Here we are finding parent component of node v
            S, c2 = self.find_comp(S, v)
            # Here we are checking if parent component of both nodes are different or not
            # If not that means if we add this edge it will going to form loop but that is not
            # possible because we are building a tree and there is no loop in tree. So we are not
            # adding that edge to connected component dictionary.
            if c1!=c2:
                # Here we are checking if c1 node is present in MST or not
                if c1 in MST:
                    MST[c1].append((c2, w))
                else:
                    MST[c1] = [(c2, w)]
                # Here we are checking if c2 node is present in MST or not
                if c2 in MST:
                    MST[c2].append((c1, w))
                else:
                    MST[c2] = [(c1, w)]
                # Here we are managing rank of the each component 
                # this part we are using to make MST building efficient.
                if R[c2] > R[c1]:
                    S[c2] = c1
                elif R[c1] > R[c2]:
                    S[c1] = c2
                else:
                    S[c2] = c1
                    R[c1] += 1
        # Here we are returning builded MST
        return MST

    def seg_alogrithm(self, G, MST, E, K):
        # This is the method where main segmentation algorithm is implemented
        # In this method we are passing MST, Adjacency List Graph and Edge List Graph
        V =  G.shape[0]
        # This list store parent component of each node, Segment Set
        S = []
        # We are using this list to store rank of each component which is part of 
        # building mst algorithm, in easy words it store how many nodes are present in
        # each connected component.
        R = []
        # Here we are initalizing default values to each list
        # In begning each node is parent of itself and rank of each component is zero
        for i in range(V):
            S.append(i)
            R.append(0)
        # Here we are sorting edge list based on edge weight in descending order.
        # So that the edges with most weight present at top and it shows there is
        # high transition between pixel intensity
        E = sorted(E, key=lambda x: x[2], reverse=True)
        # This dictionary store connected component
        comp_di = {}
        # In this loop we are iterating over each edges and building different component 
        # based on the segmentation algorithm, in the end we have a segmented image graph
        for i,edge in enumerate(E):
            sys.stdout.write('\rsegmentation completed : {0}%'.format(int((float(i)/len(E))*100)))
            u, v, w = edge
            # Here we are finding parent component of node u
            S, c1 = self.find_comp(S, u)
            # Here we are finding parent component of node v
            S, c2 = self.find_comp(S, v)
            # Here we are checking if parent component of both nodes are different or not
            # If not that means if we add this edge it will going to form loop but that is not
            # possible because we are building a tree and there is no loop in tree. So we are not
            # adding that edge to connected component dictionary.
            if c1!=c2:
                # Here we are adding nodes to each component
                comp_di = self.build_comp(comp_di, c1, c2, u, v)
                # Here we are checking if both component contain only one node
                # then merge both to one component
                if c1 == u and c2 == v:
                    # Here we are using the same rank based algorithm for building
                    # segmentation tree
                    if R[c2] > R[c1]:
                        S[c2] = c1
                    elif R[c1] > R[c2]:
                        S[c1] = c2
                    else:
                        S[c2] = c1
                        R[c1] += 1
                # If anyone or both components not contain one node then we use
                # following algorithm based on paper
                else:
                    # Here we are calculating internal difference of component c1
                    idiff_c1 = self.internal_diff(MST, comp_di[c1])
                    # Here we are calculating internal difference of component c2
                    idiff_c2 = self.internal_diff(MST, comp_di[c2])
                    # Here we are calculating tou value for each component
                    tou_c1 = K/(len(comp_di[c1])+1)
                    tou_c2 = K/(len(comp_di[c2])+1)
                    # Here we are finding minimum of internal differences of component
                    mint = min(idiff_c1+tou_c1, idiff_c2+tou_c2)
                    # Here we check that if minimum internal difference is less than current edge weight
                    # then we merge two components because this edge contain low possibility of being border
                    # pixel between two components.
                    if w > mint:
                        if R[c2] > R[c1]:
                            # Here we are merging two components
                            S[c2] = c1
                        elif R[c1] > R[c2]:
                            # Here we are merging two components
                            S[c1] = c2
                        else:
                            # Here we are merging two components
                            S[c2] = c1
                            R[c1] += 1
        # Here we are returning segmented image tree
        return S
    
    def build_comp(self, comp_di, c1, c2, u, v):
        # This method we are using to add nodes to component
        # Here we are checking if node c1 present in component dictionary or not
        if c1 in comp_di:
            # If component present in the dictionary then we add nodes to this component
            temp = [u] + comp_di[u] if u in comp_di else []
            temp = list(set(temp))
            comp_di[c1].extend(temp)
        else:
            # If component not present in the dictionary then we create a list and append
            # it to the component.
            comp_di[c1] = [u] if c1 != u else []
        if c2 in comp_di:
            # If component present in the dictionary then we add nodes to this component
            temp = [v] + comp_di[v] if v in comp_di else []
            temp = list(set(temp))
            comp_di[c2].extend(temp)
        else:
            # If component not present in the dictionary then we create a list and append
            # it to the component.
            comp_di[c2] = [v] if c2 != v else []
        return comp_di
    
    def internal_diff(self, G, C):
        # This method we are using for calculating internal difference
        # Internal difference is that edge present in component have minimum weight
        # in comparision to all the other edges in component graph.
        maxw = 10e9
        for u in C:
            if u in G:
                for v in G[u]:
                    maxw = min(v[1], maxw)
        return maxw

In [299]:
def get_color(col_di, node):
    if node in col_di:
        return col_di, col_di[node]
    else:
        b = np.random.randint(0, 255)
        g = np.random.randint(0, 255)
        r = np.random.randint(0, 255)
        col_di[node] = (b, g, r)
        return col_di, col_di[node]

#### Main Method

`Here we are creating object of GraphSegmentation class and also loading image`

In [443]:
graph_obj = GraphSegmentation()
# Here we are loading image in grayscale channel only
# img = cv2.imread('data/street.png', 0)
# img = cv2.imread('data/baseball.png', 0)
# img = cv2.imread('data/indoor.png', 0)
img = cv2.imread('data/paris.png', 0)
# img = cv2.imread('data/building.png', 0)
# img = cv2.imread('data/boy.jpg', 0)
# img = cv2.imread('data/girl.jpg', 0)
# img = cv2.imread('data/panda.jpg', 0)
# img = cv2.imread('data/people.jpg', 0)
img = cv2.resize(img, None, fx=0.7, fy=0.7)
# Here we are applying gaussian blur to image with kernel size 3 and sigma 0.8 as per paper
smooth_img = graph_obj.gaussian_filter(img, 3, 0.8)
# Here we are building graph from given image
print('[info] : building graph')
ch1_G, ch1_E = graph_obj.image_to_grid_graph(smooth_img)
print('[info] : building graph complete')

[info] : building graph
[info] : building graph complete


In [415]:
# cv2.imshow('Image', smooth_img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
# print(smooth_img.shape)

` Here we are building mst from image graph `

In [444]:
print('[info] : building mst')
MST = graph_obj.build_mst(ch1_G, ch1_E)
print('\n[info] : building mst complete')

[info] : building mst
building mst : 99%
[info] : building mst complete


`Here we are applying segmentation algorithm on image graph`

In [445]:
print('[info] : applying sementation')
S = graph_obj.seg_alogrithm(ch1_G, MST, ch1_E, 300)
print('\n[info] : segmentation complete')

[info] : applying sementation
segmentation completed : 99%
[info] : segmentation complete


`Here we are creating a blank image to store segmented components of image`

In [446]:
seg_img = np.zeros((smooth_img.shape[0], smooth_img.shape[1], 3), dtype='uint8')
col_di = {} # Segment Color Dictionary
group_di = {} # Group Dictionary
print(seg_img.shape)

(243, 324, 3)


`Here we are creating a component list, so that all the nodes from same component must present in same list`

In [447]:
for u, v in enumerate(S):
    u_row, u_col = graph_obj.vertex_to_img[u]
    S, par_v = graph_obj.find_comp(S, v)
    if par_v not in group_di:
        group_di[par_v] = [(u_row, u_col)]
    else:
        group_di[par_v].append((u_row, u_col))
#     col_di, col = get_color(col_di, par_v)
# #     col_di, col = get_color(col_di, v)
#     seg_img[u_row, u_col, :] = col

`Here I have use method to merge small component into one which is idea from selective search algorithm,
here I am using node count as merging criteria. Here we are merging all those region or component which
have number of nodes less than 5000.`
> Modified Graph Based Segmentation

In [448]:
color_coord_list = []
temp_list = []
for key in group_di:
    if len(group_di[key]) <=5000:
        temp_list.extend(group_di[key])
    else:
        color_coord_list.append(temp_list)
        temp_list = []
        color_coord_list.append(group_di[key])
color_coord_list.append(temp_list)

`Here we are assiging colors to each region or component of segmented image`

In [449]:
for coord_list in color_coord_list:
    b = np.random.randint(0, 255)
    g = np.random.randint(0, 255)
    r = np.random.randint(0, 255)
    col = (b, g, r)
    for item in coord_list:
        seg_img[item[0], item[1], :] = col

`Here we are showing segmented image output`

In [450]:
# seg_img = cv2.cvtColor(seg_img, cv2.COLOR_BGR2GRAY)
cv2.imwrite('output/1.jpg', seg_img)
cv2.imshow('Seg Image', seg_img)
cv2.waitKey(0)
cv2.destroyAllWindows()